In [1]:
import pandas as pd
import os 
import random
from tqdm import tqdm
import dtale
import matplotlib.pyplot as plt

## Load in Reddit Data

In [1]:
filepath = "data/labelled_data/"

ERROR! Session/line number was not unique in database. History logging moved to new session 61


In [2]:
all_files = os.listdir(filepath)
comments = {}
posts = {}
for fname in tqdm(all_files):
    file = pd.read_csv(filepath+fname,sep='\t').drop('Unnamed: 0',axis=1)
    if 'submission' in fname:
        posts['_'.join(fname.split('_')[3:6])] = file      
    if 'comment' in fname:
        comments['_'.join(fname.split('_')[3:6])] = file
        
        
subset = []
for date,df in posts.items():
    df['date'] = date
    subset.append(df)
    
    
merged_posts = pd.concat(subset)
subset = []
for date,df in comments.items():
    df['date'] = date
    subset.append(df)
    
merged_comments = pd.concat(subset)


cols = ['corpus','date','subreddit','conflict','similarity','knowledge','power','status','support','identity','romance','trust','fun']
posts_df = merged_posts[cols]
comments_df = merged_comments[cols]

KeyboardInterrupt: 

In [ ]:
subset = []
for date,df in posts.items():
    df['date'] = date
    subset.append(df)
    
    
merged_posts = pd.concat(subset)
subset = []
for date,df in comments.items():
    df['date'] = date
    subset.append(df)
    
merged_comments = pd.concat(subset)

In [ ]:
cols = ['corpus','date','subreddit','conflict','similarity','knowledge','power','status','support','identity','romance','trust','fun']
posts_df = merged_posts[cols]
comments_df = merged_comments[cols]

In [ ]:
posts_df

## Load in Correlation Data

In [ ]:
coin_path = "data/coins/"
coin_fnames = os.listdir(coin_path)
coin_frames = {fname.split('.')[0].split('-')[1]:pd.read_csv(coin_path+fname).drop('Unnamed: 0',axis=1) for fname in coin_fnames}
coin_frames['SOL'].head()

In [ ]:
coin_frames['ETH'].sort_values('Date').head(150).tail(50)['Date']

## Correlations

In [ ]:
should_plot = False

In [ ]:
if should_plot:
    sol_view = coin_frames['BTC'].loc[coin_frames['BTC'].Date>='2022-01-01'].sort_values('Date').head(200)
    sol_view['Mean'] = [(row['High']+row['Low'])/2 for k,row in sol_view.iterrows()]
    plt.style.use('fivethirtyeight')
    plt.figure(figsize=[16,9])
    plt.plot(sol_view['Date'],sol_view['Mean'])
    plt.fill_between(sol_view['Date'],
                     sol_view['Low'],
                     sol_view['High'],
                     alpha=.3)
    plt.show()

In [ ]:
sol_reddit = comments_df.loc[comments_df.subreddit == 'Bitcoin']
sol_reddit.head()

In [ ]:
sol_reddit = sol_reddit.sort_values('date')
sol_agg_mean = sol_reddit[sol_reddit.columns[1:]].groupby("date").median().reset_index()
#sol_agg_mean = sol_ss
sol_agg_std = sol_reddit.groupby("date").std()

In [ ]:
dfs = {}

for col in sol_agg_std.columns:
    dim_dict = {'date':sol_agg_std.index.tolist(),
                'mean':sol_agg_mean[col].values.tolist(),
                'min':list((sol_agg_mean[col].values-sol_agg_std[col].values)),
                'max':list((sol_agg_mean[col].values+sol_agg_std[col].values))}
    
    
    
    
    dfs[col] = pd.DataFrame(dim_dict)

In [ ]:
#df_nat['new_cases'].rolling(7).sum()
sol_ss = sol_reddit[sol_reddit.columns[1:]]
for col in dfs.keys():
    st = col
    sol_ss[st] = sol_ss[col].rolling(7).mean()

In [ ]:
sol_agg_std

In [19]:
#sol_agg_std.index.tolist()
#sol_agg_mean[col].values.tolist()

#sol_agg_mean[col].values
#sol_agg_std[col].values

#sol_agg_mean

In [18]:
if should_plot:
    for dim in sol_agg_mean.columns[1:]:
        plt.style.use('fivethirtyeight')
        plt.figure(figsize=[16,9])
        plt.title(dim)
        plt.plot(dfs[dim]['date'],
                 dfs[dim]['mean'])

        plt.fill_between(dfs[dim]['date'],
                         dfs[dim]['min'],
                         dfs[dim]['max'],
                         alpha=.3)

        plt.show()

dtale.show(pd.DataFrame({dim:dfs[dim]['mean'] for dim in dfs.keys()}))

## Create Bitcoin Dataset

In [19]:
all_files = os.listdir(filepath)
comments = {}
posts = {}
for fname in tqdm(all_files):
    file = pd.read_csv(filepath+fname,sep='\t').drop('Unnamed: 0',axis=1)
    if 'submission' in fname:
        posts['_'.join(fname.split('_')[3:6])] = file      
    if 'comment' in fname:
        comments['_'.join(fname.split('_')[3:6])] = file
        
        
subset = []
for date,df in posts.items():
    df['date'] = date
    df['text_type'] = "post" 
    subset.append(df)
    
    
merged_posts = pd.concat(subset)
subset = []
for date,df in comments.items():
    df['date'] = date
    df['text_type'] = "comment" 
    subset.append(df)
    
merged_comments = pd.concat(subset)


cols = ['corpus','date','text_type','subreddit','conflict','similarity','knowledge','power','status','support','identity','romance','trust','fun']
posts_df = merged_posts[cols]
comments_df = merged_comments[cols]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 590/590 [00:09<00:00, 59.98it/s]


In [20]:
coin_path = "data/coins/"
coin_fnames = os.listdir(coin_path)
coin_frames = {fname.split('.')[0].split('-')[1]:pd.read_csv(coin_path+fname).drop('Unnamed: 0',axis=1) for fname in coin_fnames}

BTC_prices = coin_frames['BTC']
BTC_prices['Date'] = BTC_prices['Date'].apply(lambda x: x.replace('-','_'))

BTC_prices['Mean'] = (BTC_prices['High']+BTC_prices['Low'])/2
BTC_prices

,Date,Open,High,Low,Close,Volume,Market Cap,Mean
0,2022_09_18,20127.233948,20127.233948,19387.491348,19419.506191,3.125478e+10,3.719704e+11,19757.362648
1,2022_09_17,19777.034028,20162.532022,19777.034028,20127.576657,2.495745e+10,3.855133e+11,19969.783025
2,2022_09_16,19704.005441,19870.629766,19400.075449,19772.584206,3.012336e+10,3.786932e+11,19635.352608
3,2022_09_15,20242.289908,20318.166392,19636.733774,19701.209963,3.638901e+10,3.773113e+11,19977.450083
4,2022_09_14,20184.555325,20467.201433,19793.396718,20241.089802,3.787238e+10,3.876319e+11,20130.299075
...,...,...,...,...,...,...,...,...
3426,2013_05_02,116.379997,125.599998,92.281898,105.209999,0.000000e+00,1.168517e+09,108.940948
3427,2013_05_01,139.000000,139.889999,107.720001,116.989998,0.000000e+00,1.298955e+09,123.805000
3428,2013_04_30,144.000000,146.929993,134.050003,139.000000,0.000000e+00,1.542813e+09,140.489998
3429,2013_04_29,134.444000,147.488007,134.000000,144.539993,0.000000e+00,1.603769e+09,140.744003


In [21]:
sources = ['posts','comments','combined']
source_map = {'posts':posts_df,
              'comments':comments_df,
              'combined':pd.concat([posts_df,comments_df])}

source_choice = sources[-1]



aggregation = 'median'


#cols = ['corpus','date','conflict','similarity','knowledge','power','status','support','identity','romance','trust','fun']

source_df = source_map[source_choice]

btc_reddit = source_df.loc[source_df.subreddit == 'Bitcoin'][cols].sort_values('date')
btc_reddit

,corpus,date,text_type,subreddit,conflict,similarity,knowledge,power,status,support,identity,romance,trust,fun
108,invest in cryptocurrency using paxful removed,2022_01_02,post,Bitcoin,0.428484,0.429621,0.693272,0.587397,0.347396,0.433853,0.503711,0.082732,5.618975e-01,0.262231
66,in honor of hal finney the inaugural running b...,2022_01_02,post,Bitcoin,0.128358,0.006815,0.989936,0.039752,0.197273,0.304415,0.320837,0.001410,8.298105e-07,0.264381
65,in honor of hal finney the inaugural running b...,2022_01_02,post,Bitcoin,0.316497,0.186380,0.651785,0.190209,0.386639,0.350347,0.389105,0.011938,5.916070e-01,0.469555
64,stolen seed i have a question regarding people...,2022_01_02,post,Bitcoin,0.882927,0.007469,0.988437,0.024338,0.128773,0.002562,0.160387,0.003984,8.469893e-07,0.208423
63,bitcoin prototype bit gold by nick szabo 1998 ...,2022_01_02,post,Bitcoin,0.330860,0.085265,0.868963,0.090066,0.273570,0.175537,0.472902,0.006401,3.738376e-05,0.369482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,remember that embarrassing thing in high schoo...,2022_11_09,comment,Bitcoin,0.085449,0.783484,0.619302,0.015285,0.113561,0.326193,0.389684,0.080402,6.451114e-01,0.713825
118,that s why i spend everything i can on bitcoin...,2022_11_09,comment,Bitcoin,0.115963,0.194062,0.948707,0.017420,0.066770,0.173337,0.183617,0.002718,8.337976e-07,0.423309
117,using qr ingrained gold leafs as physical tran...,2022_11_09,comment,Bitcoin,0.375630,0.289607,0.941603,0.032308,0.187667,0.032184,0.416249,0.013707,8.556129e-07,0.116874
126,btc is a fancy lottery ticket it s either goin...,2022_11_09,comment,Bitcoin,0.418905,0.008516,0.988385,0.054717,0.075944,0.003615,0.530918,0.058263,8.458398e-07,0.426632


In [22]:
btc_full = pd.merge(btc_reddit,BTC_prices[['Date','High','Low','Mean']],left_on='date',right_on='Date').drop('Date',axis=1)
btc_full.set_index('date',inplace=True)

In [23]:
event_dict = {'eventID':[1,2,3,4,5,6],
              
             'before_start':['2022-03-05','2022-01-27','2022-02-23','2022-06-01','2022-04-27','2022-01-12'],
             'before_end':['2022-03-13','2022-02-03','2022-02-27','2022-06-07','2022-05-04','2022-01-18'],
              
             'during_start':['2022-03-13','2022-02-03','2022-02-27','2022-06-07','2022-05-04','2022-01-18'],
             'during_end':['2022-03-28','2022-02-09','2022-03-01','2022-06-18','2022-05-12','2022-01-22'],
              
             'after_start':['2022-03-28','2022-02-09','2022-03-01','2022-06-18','2022-05-12','2022-01-22'],
             'after_end':['2022-04-04','2022-02-16','2022-03-07','2022-07-01','2022-05-19','2022-01-30'],
    
             'event_type':['positive','positive','positive','negative','negative','negative']}



event_frame = pd.DataFrame(event_dict)
event_frame

,eventID,before_start,before_end,during_start,during_end,after_start,after_end,event_type
0,1,2022-03-05,2022-03-13,2022-03-13,2022-03-28,2022-03-28,2022-04-04,positive
1,2,2022-01-27,2022-02-03,2022-02-03,2022-02-09,2022-02-09,2022-02-16,positive
2,3,2022-02-23,2022-02-27,2022-02-27,2022-03-01,2022-03-01,2022-03-07,positive
3,4,2022-06-01,2022-06-07,2022-06-07,2022-06-18,2022-06-18,2022-07-01,negative
4,5,2022-04-27,2022-05-04,2022-05-04,2022-05-12,2022-05-12,2022-05-19,negative
5,6,2022-01-12,2022-01-18,2022-01-18,2022-01-22,2022-01-22,2022-01-30,negative


In [24]:
event_frame.head()

,eventID,before_start,before_end,during_start,during_end,after_start,after_end,event_type
0,1,2022-03-05,2022-03-13,2022-03-13,2022-03-28,2022-03-28,2022-04-04,positive
1,2,2022-01-27,2022-02-03,2022-02-03,2022-02-09,2022-02-09,2022-02-16,positive
2,3,2022-02-23,2022-02-27,2022-02-27,2022-03-01,2022-03-01,2022-03-07,positive
3,4,2022-06-01,2022-06-07,2022-06-07,2022-06-18,2022-06-18,2022-07-01,negative
4,5,2022-04-27,2022-05-04,2022-05-04,2022-05-12,2022-05-12,2022-05-19,negative


In [25]:
btc_full.drop('subreddit',axis=1,inplace=True)

In [26]:
btc_full.head()

,corpus,text_type,conflict,similarity,knowledge,power,status,support,identity,romance,trust,fun,High,Low,Mean
date,,,,,,,,,,,,,,,
2022_01_02,invest in cryptocurrency using paxful removed,post,0.428484,0.429621,0.693272,0.587397,0.347396,0.433853,0.503711,0.082732,5.618975e-01,0.262231,47881.406233,46856.93631,47369.171271
2022_01_02,in honor of hal finney the inaugural running b...,post,0.128358,0.006815,0.989936,0.039752,0.197273,0.304415,0.320837,0.001410,8.298105e-07,0.264381,47881.406233,46856.93631,47369.171271
2022_01_02,in honor of hal finney the inaugural running b...,post,0.316497,0.186380,0.651785,0.190209,0.386639,0.350347,0.389105,0.011938,5.916070e-01,0.469555,47881.406233,46856.93631,47369.171271
2022_01_02,stolen seed i have a question regarding people...,post,0.882927,0.007469,0.988437,0.024338,0.128773,0.002562,0.160387,0.003984,8.469893e-07,0.208423,47881.406233,46856.93631,47369.171271
2022_01_02,bitcoin prototype bit gold by nick szabo 1998 ...,post,0.330860,0.085265,0.868963,0.090066,0.273570,0.175537,0.472902,0.006401,3.738376e-05,0.369482,47881.406233,46856.93631,47369.171271


In [27]:
btc_full.reset_index(inplace=True)
btc_full['date'] = btc_full['date'].apply(lambda x: x.replace('_','-'))

In [28]:
periods = []

for k,row in tqdm(event_frame.iterrows()):

  
    
    before = btc_full.loc[(btc_full['date']>row['before_start'])&(btc_full['date']<=row['before_end'])]
    if before.shape[0]:
        before['event_id'] = [row['eventID']]*before.shape[0]
        before['event_type'] = [row['event_type']]*before.shape[0]
        before['period_type'] = ['before']*before.shape[0]
        periods.append(before)
        
        
    during = btc_full.loc[(btc_full['date']>row['during_start'])&(btc_full['date']<=row['during_end'])]
    if during.shape[0]:
        during['event_id'] = [row['eventID']]*during.shape[0]
        during['event_type'] = [row['event_type']]*during.shape[0]
        during['period_type'] = ['during']*during.shape[0]
        periods.append(during)
                       
                       
    after = btc_full.loc[(btc_full['date']>row['after_start'])&(btc_full['date']<=row['after_end'])]
    if after.shape[0]:
        after['event_id'] = [row['eventID']]*after.shape[0]
        after['event_type'] = [row['event_type']]*after.shape[0]
        after['period_type'] = ['after']*after.shape[0]
        periods.append(after)
    

df_temp = pd.concat(periods)    
print(df_temp.shape)

0it [00:00, ?it/s]/var/folders/b4/cwqn8hyj67100247892j531w0000gn/T/ipykernel_4481/3146999682.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/b4/cwqn8hyj67100247892j531w0000gn/T/ipykernel_4481/3146999682.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/b4/cwqn8hyj67100247892j531w0000gn/T/ipykernel_4481/3146999682.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

(42710, 19)


In [29]:
boring = btc_full.loc[~btc_full.date.isin(df_temp.date.values)]
boring['period_type'] = [None]*boring.shape[0]
boring['event_id'] = [None]*boring.shape[0]
boring['event_type'] = [None]*boring.shape[0]
final = pd.concat([df_temp,boring])

final

/var/folders/b4/cwqn8hyj67100247892j531w0000gn/T/ipykernel_4481/2711685107.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/b4/cwqn8hyj67100247892j531w0000gn/T/ipykernel_4481/2711685107.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/b4/cwqn8hyj67100247892j531w0000gn/T/ipykernel_4481/2711685107.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,date,corpus,text_type,conflict,similarity,knowledge,power,status,support,identity,romance,trust,fun,High,Low,Mean,event_id,event_type,period_type
21948,2022-03-06,sarn sauron coin bsc gem of 2022 launch now e...,post,0.217982,0.008616,0.987545,0.095460,0.210643,0.012049,0.401652,0.004092,8.436824e-07,0.404854,39640.176751,38211.647962,38925.912357,1,positive,before
21949,2022-03-06,a proposal to you removed,post,0.503045,0.435572,0.485023,0.518171,0.427520,0.495435,0.432057,0.254124,3.306695e-01,0.453298,39640.176751,38211.647962,38925.912357,1,positive,before
21950,2022-03-06,time to buy nan,post,0.315845,0.453668,0.491014,0.467255,0.453663,0.485220,0.443327,0.571401,4.674791e-01,0.595824,39640.176751,38211.647962,38925.912357,1,positive,before
21951,2022-03-06,stupid question please excuse my ignorance can...,post,0.685344,0.003814,0.995469,0.033478,0.095750,0.000713,0.207318,0.000114,8.855732e-07,0.099581,39640.176751,38211.647962,38925.912357,1,positive,before
21952,2022-03-06,bitcoin nan,post,0.488357,0.469591,0.483014,0.434496,0.515467,0.484323,0.482162,0.507920,5.038089e-01,0.523179,39640.176751,38211.647962,38925.912357,1,positive,before
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75521,2022-09-18,hi u alfador8 thanks for tipping u cryptoenthu...,comment,0.730620,0.003401,0.995973,0.980206,0.151547,0.000652,0.381715,0.000014,8.378324e-07,0.052759,20127.233948,19387.491348,19757.362648,None,None,None
75522,2022-09-18,i m very new,comment,0.221113,0.536141,0.441375,0.422223,0.597648,0.516809,0.502142,0.645875,6.132007e-01,0.539723,20127.233948,19387.491348,19757.362648,None,None,None
75523,2022-09-18,gt a purely peer to peer version of electroni...,comment,0.395776,0.093113,0.985907,0.042485,0.083406,0.026649,0.297737,0.009478,8.965515e-01,0.121190,20127.233948,19387.491348,19757.362648,None,None,None
75524,2022-09-18,the house is 87k how am i going to buy a a 94k...,comment,0.363920,0.168305,0.738788,0.239386,0.104419,0.204042,0.385997,0.012725,1.151722e-06,0.593254,20127.233948,19387.491348,19757.362648,None,None,None


In [60]:
final.to_csv('aggregated_dataset.tsv',index=None,sep=';')

In [46]:
btc_full.loc[(btc_full.index>'2022-01-22')&(btc_full.index<='2022-01-30')]

,corpus,text_type,conflict,similarity,knowledge,power,status,support,identity,romance,trust,fun,High,Low,Mean
date,,,,,,,,,,,,,,,


In [62]:
!warp

zsh:1: command not found: warp


In [295]:
event_id = []
event_type = []
period_type = []

for k,row in tqdm(btc_full.iterrows()):
    not_app = False
    for k2, row2 in event_frame.iterrows():
        if (k>row2['before_start']) and (k<=row2['before_end']):
            event_id.append(row2['eventID'])
            event_type.append(row2['event_type'])
            period_type.append('before')
            not_app=True
        elif (k>row2['during_start']) and (k<=row2['during_end']):
            event_id.append(row2['eventID'])
            event_type.append(row2['event_type'])
            period_type.append('during')
            not_app=True
            
        elif (k>row2['after_end']) and (k<=row2['after_end']):
            event_id.append(row2['eventID'])
            event_type.append(row2['event_type'])
            period_type.append('end') 
            not_app=True
    
    if not_app:

        event_id.append(None)
        event_type.append(None)
        period_type.append(None)
            
            
btc_full['event_id'] = event_id
btc_full['event_type'] = event_type
btc_full['period_type'] = period_type
btc_full

75526it [00:35, 2127.41it/s]


ValueError: Length of values (0) does not match length of index (75526)

In [ ]:
btc_full.shape